In [19]:
import numpy as np
import altair as alt
import pandas as pd
import database as db

query = """
SELECT p.year, p.day, p.star, p.time as personal, s.time as global, p.position as position
FROM personal p
JOIN scores s
ON p.year = s.year AND p.day = s.day AND p.star = s.star
WHERE s.position = 100
"""

conn = db.open_db('aoc.db')
df = db.do_df(conn, query)
df['personal'] /= 60
df['global'] /= 60
df['relative_time'] = df['personal'] / df['global']
print('personal stats imported from the following ', df.value_counts('year'))

personal stats imported from the following  year
2020    50
2021    50
2017    46
dtype: int64


In [2]:
selection = alt.selection_multi(fields=['year'])

color = alt.condition(selection,
                      alt.Color('year:N', legend=None),
                      alt.value('lightgray'))

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(type='log', domain=(1, 400))),
    y=alt.Y('global:Q',scale=alt.Scale(type='log', domain=(1,170))),
    color = color,
    tooltip=['year','day','star']
).properties(
    title='Personal time vs global time',
    width=600,
    height=600)


legend = alt.Chart(df).mark_point().encode(
    y=alt.Y('year:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

chart | legend

alt.HConcatChart(...)

In [3]:
df

,year,day,star,personal,global,position,relative_time
0,2017,3,2,52.000000,23.316667,NaN,2.230164
1,2017,3,1,15.000000,8.483333,NaN,1.768173
2,2017,4,2,20.000000,3.666667,NaN,5.454545
3,2017,4,1,2.000000,1.883333,NaN,1.061947
4,2017,5,2,22.000000,4.766667,NaN,4.615385
...,...,...,...,...,...,...,...
141,2021,23,1,184.066667,13.816667,2768.0,13.322075
142,2021,24,2,214.650000,76.750000,1263.0,2.796743
143,2021,24,1,214.100000,70.466667,1357.0,3.038316
144,2021,25,2,18.933333,9.566667,629.0,1.979094


In [4]:
dom = 'log'
chart = alt.Chart(df).mark_point().encode(
    x=alt.X('personal:Q',scale=alt.Scale(type=dom, domain=(1, 400))),
    y=alt.Y('relative_time:Q',scale=alt.Scale(type=dom, domain=(1,30))),
    column = 'year:N',
    row = 'star:O',
    tooltip=['year','day','star']
)
chart 

alt.Chart(...)

In [7]:
# the regression didn't really have a good fit
b  = chart.transform_regression('personal', 'global', params=True, groupby=['year'], method='linear').mark_line()
import altair_transform
print('regressions for all the years, r2 close to 1 is good')
print(altair_transform.extract_data(b))

regressions for all the years, r2 close to 1 is good
                                       coef  rSquared    keys
0  [5.549883440888063, 0.23786632715557612]  0.655115  [2017]
1  [6.283584844950919, 0.08694103057576778]  0.464877  [2020]
2  [5.075090859854178, 0.20660989060530094]  0.710581  [2021]


In [18]:
# df.groupby('Category').agg({'Item':'size','shop1':['sum','mean','std'],'shop2':['sum','mean','std'],'shop3':['sum','mean','std']})
stats = ['median', 'mean', 'max', 'min']
df2 = df.groupby('year').agg({'global':stats, 'personal':stats, 'relative_time':stats, 'position':stats }).T.reset_index()
df2 = df2.fillna(-1)
for i in range(2,df2.shape[1]):
    df2.iloc[:,i] = df2.iloc[:,i].astype(int)
df2

year,level_0,level_1,2017,2020,2021
0,global,median,11,8,9
1,global,mean,15,11,16
2,global,max,54,73,76
3,global,min,1,2,1
4,personal,median,29,22,20
5,personal,mean,40,57,57
6,personal,max,193,402,360
7,personal,min,2,4,1
8,relative_time,median,2,2,2
9,relative_time,mean,2,4,2
